Datenterm: $$\sum_{(i,j)\in k}(r_{ij}-\vec{p}_i^T\vec{q}_j)^2$$
Matrixdarstellung: $$||R-P Q^T||_2^2$$

In [7]:
import numpy as np

def cost(P, Q, R, lambda_, np, nq):
    data_term = np.linalg.norm(R - P @ Q.T)**2
    regularization_p = np.sum(np * np.linalg.norm(P)**2)
    regularization_q = np.sum(nq * np.linalg.norm(Q)**2)
    regularization_term = lambda_ * (regularization_p + regularization_q)
    return data_term + regularization_term

In [8]:
def solveBlock_P(R, Q, I, lambda_, n_p):
    m, n = R.shape
    k = Q.shape[1]
    P = np.zeros((m, k))
    I = I.astype(bool)

    for i in range(m):
        I_i = I[i, :]
        Q_i = Q[I_i, :]
        R_i = R[i, I_i]

        A = Q_i.T @ Q_i + lambda_ * n_p[i] * np.eye(k)
        b = Q_i.T @ R_i
        P[i, :] = np.linalg.solve(A, b)
    return P

def solveBlock_Q(R, P, I, lambda_, n_q):
    m, n = R.shape
    k = P.shape[1]
    Q = np.zeros((n, k))
    I = I.astype(bool)

    for j in range(n):
        I_j = I[:, j]
        P_j = P[I_j, :]
        R_j = R[I_j, j]

        A = P_j.T @ P_j + lambda_ * n_q[j] * np.eye(k)
        b = P_j.T @ R_j
        Q[j, :] = np.linalg.solve(A, b)
    return Q

In [16]:
def calc_rmse(R, I, P, Q):
    R_pred = P @ Q.T
    error = I * (R - R_pred)
    rmse = np.sqrt(np.sum(error**2) / np.sum(I))
    return rmse

def alternating_least_squares(R, I, k=3, lambda_=0.1, max_iter=100, tolerance=1e-4):
    m, n = R.shape
    P = np.random.rand(m, k)
    Q = np.random.rand(n, k)
    n_p = np.ones(m)
    n_q = np.ones(n)
    rmse_history = []

    for iteration in range(max_iter):
        P = solveBlock_P(R, Q, I, lambda_, n_p)
        Q = solveBlock_Q(R, P, I, lambda_, n_q)
        
        rmse = calc_rmse(R, I, P, Q)
        rmse_history.append(rmse)
        
        print(f"Iteration {iteration + 1}/{max_iter}, RMSE: {rmse:.6f}")
        
        if iteration > 0 and abs(rmse_history[-1] - rmse_history[-2]) < tolerance:
            print("convergence reached")
            break

    return P, Q, rmse_history

In [19]:
from utility import setup_dataset, visualize_sparsity_pattern

R, I = setup_dataset(toy_ds=True)
P, Q, rmse_history = alternating_least_squares(R, I, k=3, lambda_=0.1, max_iter=50)
P, Q

Iteration 1/50, RMSE: 0.040180
Iteration 2/50, RMSE: 0.037191
Iteration 3/50, RMSE: 0.035600
Iteration 4/50, RMSE: 0.035507
convergence reached


(array([[ 3.45076918,  0.98061268, -0.42105107],
        [ 2.99715646,  2.45869326, -1.98619753],
        [ 1.2949067 ,  0.39992246,  3.10252177],
        [ 1.23783583, -0.35597315,  2.42908326]]),
 array([[ 1.3544459 ,  0.1750684 , -0.25958536],
        [ 0.90988533,  0.95361675,  0.06834295],
        [ 0.94213165,  0.21126922,  1.18469122]]))

In [20]:
R, I = setup_dataset(toy_ds=False)
P, Q, rmse_history = alternating_least_squares(R, I, k=10, lambda_=0.1, max_iter=100)

Iteration 1/100, RMSE: 0.829587
Iteration 2/100, RMSE: 0.748141
Iteration 3/100, RMSE: 0.718921
Iteration 4/100, RMSE: 0.704069
Iteration 5/100, RMSE: 0.695263
Iteration 6/100, RMSE: 0.689308
Iteration 7/100, RMSE: 0.684942
Iteration 8/100, RMSE: 0.681652
Iteration 9/100, RMSE: 0.679154
Iteration 10/100, RMSE: 0.677184
Iteration 11/100, RMSE: 0.675567
Iteration 12/100, RMSE: 0.674229
Iteration 13/100, RMSE: 0.673113
Iteration 14/100, RMSE: 0.672158
Iteration 15/100, RMSE: 0.671316
Iteration 16/100, RMSE: 0.670556
Iteration 17/100, RMSE: 0.669860
Iteration 18/100, RMSE: 0.669224
Iteration 19/100, RMSE: 0.668641
Iteration 20/100, RMSE: 0.668107
Iteration 21/100, RMSE: 0.667614
Iteration 22/100, RMSE: 0.667153
Iteration 23/100, RMSE: 0.666720
Iteration 24/100, RMSE: 0.666312
Iteration 25/100, RMSE: 0.665929
Iteration 26/100, RMSE: 0.665573
Iteration 27/100, RMSE: 0.665243
Iteration 28/100, RMSE: 0.664935
Iteration 29/100, RMSE: 0.664649
Iteration 30/100, RMSE: 0.664389
Iteration 31/100, R